In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from keras import backend as K
from keras import applications
from keras.models import Sequential,save_model,load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers
% matplotlib inline

C:\Users\AASHISH\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
img_width, img_height = 256, 256
train_data_dir = "data/train/"
validation_data_dir = "data/test/"
nb_train_samples = 4125
nb_validation_samples = 466
batch_size = 16
epochs = 3
output_num_classes = 2

In [7]:
K.set_image_data_format('channels_last')
print(K.image_data_format())

channels_last


In [8]:
model = applications.VGG19(weights = 'imagenet',
                          include_top = False,
                          input_shape = (img_width, img_height, 3))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
bloc

In [9]:
for layer in model.layers[:5]:
    layer.trainable = False

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [11]:
from keras.models import Model

x = model.output
x = Flatten()(x)
x = Dense(1024, activation = 'relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation = 'relu')(x)
preditions = Dense(output_num_classes,
                   activation = 'softmax')(x)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
model_final = Model(input = model.input, output = preditions)

C:\Users\AASHISH\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [13]:

model_final.compile(loss = "categorical_crossentropy",
                   optimizer = optimizers.SGD(lr=0.0001,momentum=0.9),
                    metrics=['accuracy'])

In [14]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip=True)
train_generatore = train_datagen.flow_from_directory(train_data_dir,
                                                     target_size=(img_width, img_height),# if deffer the resize it himself
                                                     batch_size=batch_size,
                                                     class_mode='categorical'
                                                     )



# validation_datagen = ImageDataGenerator(rescale=1./255)                                  
# validation_generatore = validation_datagen.flow_from_directory(validation_data_dir,
#                                                      target_size=(img_width, img_height),# if deffer the resize it himself
#                                                      batch_size=batch_size,
#                                                      class_mode='categorical'
#                                                      )

Found 25000 images belonging to 2 classes.


In [15]:
checkpoint = ModelCheckpoint('vgg.h5', #highest accuary of having weights 
                             monitor = 'val_acc',
                             save_best_only = True,
                             save_weights_only = False,
                             mode = 'auto',
                             verbose = 1,
                             period = 1)

In [16]:
early = EarlyStopping(monitor='val_acc',
                      min_delta = 0,
                      patience = 10,
                      verbose = 1,
                      mode = 'auto'
                      )

In [17]:
model_final.fit_generator(train_generatore,
#                           validation_data = validation_generatore,
                          epochs = epochs,
                          samples_per_epoch=nb_train_samples,
                          nb_val_samples = nb_validation_samples,
                          callbacks = [checkpoint, early])

model_final.save_weights("model/gen_first_try.h5")


Instructions for updating:
Use tf.cast instead.


C:\Users\AASHISH\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
C:\Users\AASHISH\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=3, callbacks=[<keras.ca..., steps_per_epoch=257, validation_steps=466)`
  


Epoch 1/3
257/257 [==============================] - 9991s 39s/step - loss: 0.4762 - acc: 0.7554
Epoch 2/3


C:\Users\AASHISH\Anaconda3\lib\site-packages\keras\callbacks.py:434: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
C:\Users\AASHISH\Anaconda3\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


257/257 [==============================] - 10162s 40s/step - loss: 0.1744 - acc: 0.9280
Epoch 3/3
257/257 [==============================] - 9556s 37s/step - loss: 0.1260 - acc: 0.9470


AttributeError: 'Model' object has no attribute 'save_model'

In [19]:
model_final.save("model/entire_model.h5")

In [2]:
model_final = load_model("model/entire_model.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [3]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [116]:
predict_datagen = ImageDataGenerator(rescale=1./255)                                  
predict_generatore = predict_datagen.flow_from_directory("data/test/",
                                                     target_size=(img_width, img_height),# if deffer the resize it himself
                                                     class_mode='categorical',
                                                     batch_size = 1,
                                                     shuffle = False
                                                     )

filenames = predict_generatore.filenames
# print(filenames)

nb_samples = len(filenames)
print(nb_samples)


Found 12500 images belonging to 1 classes.
12500


In [108]:
pred = model_final.predict_generator(predict_generatore, steps=nb_samples)

In [ ]:
print(pred)